In [ ]:
import tensorflow as tf 
import os
from random import shuffle

# Loading data and navigating to proper directory
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


1. Loading Data and Preprocessing

In [ ]:
# 20% Validation Set, 80% Training Set
# Input data is balanced across the number of fish classes
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input, # Preprocessing function
    validation_split=0.2 
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input # Preprocessing function
)

In [1]:
# Shuffle = True randomly selects images from a random directory/class to meet the streaming batch size and send to the model for training
# Instead of flow_from_directory, the following article: https://www.kaggle.com/pavfedotov/fish-classifier-efficientnet-acc-100, uses flow_from_dataframe
# which simply contains the list of all image paths in directory and the corresponding class label, we can pivot to this method if it is difficult
# to visualize results, but the method below is actually more efficient...
train_images = train_generator.flow_from_directory(
    directory= 'DataLoadTest',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32, # Will stream 32 images at a time to the model, helps to reduce RAM requirements
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_directory(
    directory= 'DataLoadTest',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation' # Will only take 20% of the total data as the validation data
)

# test_images = test_generator.flow_from_directory(
#     directory= 'Actual Test Dir',
#     target_size=(224, 224),
#     color_mode='rgb',
#     class_mode='categorical',
#     batch_size=32,
#     shuffle=True # I wonder why this was originally set to false
# )

NameError: ignored

In [ ]:
print("Training image shape:", train_images.image_shape)
print("Validation image shape:", val_images.image_shape)

Training image shape: (224, 224, 3)
Validation image shape: (224, 224, 3)


In [ ]:
train_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt Head Bream': 1,
 'Horse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [ ]:
val_images.class_indices

{'Black Sea Sprat': 0,
 'Gilt Head Bream': 1,
 'Horse Mackerel': 2,
 'Red Mullet': 3,
 'Red Sea Bream': 4,
 'Sea Bass': 5,
 'Shrimp': 6,
 'Striped Red Mullet': 7,
 'Trout': 8}

In [ ]:
train_images.filenames

['Black Sea Sprat/00011.png',
 'Black Sea Sprat/00012.png',
 'Black Sea Sprat/00013.png',
 'Black Sea Sprat/00014.png',
 'Black Sea Sprat/00015.png',
 'Black Sea Sprat/00016.png',
 'Black Sea Sprat/00017.png',
 'Black Sea Sprat/00018.png',
 'Black Sea Sprat/00019.png',
 'Black Sea Sprat/00020.png',
 'Black Sea Sprat/00021.png',
 'Black Sea Sprat/00022.png',
 'Black Sea Sprat/00023.png',
 'Black Sea Sprat/00024.png',
 'Black Sea Sprat/00025.png',
 'Black Sea Sprat/00026.png',
 'Black Sea Sprat/00027.png',
 'Black Sea Sprat/00028.png',
 'Black Sea Sprat/00029.png',
 'Black Sea Sprat/00030.png',
 'Black Sea Sprat/00031.png',
 'Black Sea Sprat/00032.png',
 'Black Sea Sprat/00033.png',
 'Black Sea Sprat/00034.png',
 'Black Sea Sprat/00035.png',
 'Black Sea Sprat/00036.png',
 'Black Sea Sprat/00037.png',
 'Black Sea Sprat/00038.png',
 'Black Sea Sprat/00039.png',
 'Black Sea Sprat/00040.png',
 'Black Sea Sprat/00041.png',
 'Black Sea Sprat/00042.png',
 'Black Sea Sprat/00043.png',
 'Black Se

In [ ]:
import keras
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
import numpy as np
from keras.callbacks import ModelCheckpoint, EarlyStopping

2. Defining VGG16 (CNN) Architecture

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=9, activation="softmax")) # Match number of fish classes/species (was 2 before - caused graph error)

optimizer = Adam(learning_rate=0.001) # Fine tune
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                    

3. Defining Schedulers and Callbacks

In [ ]:
model_name = "best_model_vgg_cnn.h5"
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 20) # Fine tune

monitor = tf.keras.callbacks.ModelCheckpoint(model_name, monitor='val_loss',\
                                             verbose=0,save_best_only=True,\
                                             save_weights_only=True,\
                                             mode='min') # Fine tune (we may want different saving parameters)
# Learning rate schedule
def scheduler(epoch, lr): # Fine tune
    if epoch%10 == 0: 
        lr = lr/2 
    return lr

lr_schedule = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose = 0)

4. Training Model

In [ ]:
model.fit(
    train_images, # Generator (contains class labels)
    validation_data=val_images, # Generator (contains class labels)
    batch_size=32, # Fine tune
    epochs=10, # Fine tune
    callbacks=[early_stop, monitor, lr_schedule]
)
model.fit(train_images,batch_size = 32, epochs = 12, \
          verbose = 1, callbacks= [early_stop, monitor, lr_schedule],validation_data=val_images) # Fine tune

KeyboardInterrupt: ignored

Deployment opportunities, once hyperparameters are finalized:
1. DDW DL Bundle K80 GPU
2. DDW DL Bundle V100 GPU
3. TALC Cluster - bleh